In [8]:
%pip install pandas
%pip install numpy
%pip install seaborn
%pip install mathplotlib
%pip install nltk
%pip install scikit-learn
%pip install wordcloud
%pip install textblob
%pip install spacy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement mathplotlib (from versions: none)
ERROR: No matching distribution found for mathplotlib
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
import spacy
import re, string, unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import os
import warnings

In [10]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dsdevnull/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
imdb_data = pd.read_csv("./data/IMDB Dataset.csv")

print(imdb_data.shape)
imdb_data.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


Seems like this is a relatively simple dataset, 2 columns and 50k rows. We will need to do some data cleaning, if you notice the HTML tags present in the sample data.


In [12]:
imdb_data.describe()
# Lots of unique values which is awesome for training a model!

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [13]:
imdb_data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

# Data Cleaning

I decided to use regex, I saw some examples using beautiful soup, for the time being I think that is overkill and introduces dependecies that are not needed.


In [14]:
def clean_html(line) -> string:
    clean_data = re.compile("<.*>")
    return re.sub(clean_data, "", line)


def remove_spec_char(line) -> string:
    pattern = r"[^a-zA-z0-9\s]"
    return re.sub(pattern, "", line)

In [15]:
imdb_data["review"] = imdb_data["review"].apply(clean_html)
imdb_data["review"] = imdb_data["review"].apply(remove_spec_char)
imdb_data["review"] = imdb_data["review"].str.lower()

In [16]:
imdb_data.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production the realism real...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically theres a family where a little boy j...,negative
4,petter matteis love in the time of money is a ...,positive


Text Stemming is something that I have not used before, it was interesting to read about it. I did some research on it and why it is important in NLP specifcally.


In [17]:
# This function takes a while to run depending on your PC Specs
# It took about 2m 10s on my 2018 13in MacBook Pro
def simple_stemmer(text):
    ps = PorterStemmer()
    text = " ".join([ps.stem(word) for word in text.split()])
    return text


imdb_data["review"] = imdb_data["review"].apply(simple_stemmer)

# Tokenization and removing stopwords

Stop words do not provide meaning to understanding the meaning of a given statement. Below you can see an exmaple of stop words


In [18]:
# Creating tokenizer for later use
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words("english")
print(stopword_list)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [19]:
def remove_stop_words(line, is_lower_case=False) -> string:
    tokens = tokenizer.tokenize(line)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [
            token for token in tokens if token.lower() not in stopword_list
        ]
    filtered_text = " ".join(filtered_tokens)
    return filtered_text


imdb_data["review"] = imdb_data["review"].apply(remove_stop_words)
print(imdb_data.head())

                                              review sentiment
0  one review ha mention watch 1 oz episod youll ...  positive
1  wonder littl product realism realli come home ...  positive
2  thought thi wa wonder way spend time hot summe...  positive
3  basic famili littl boy jake think zombi hi clo...  negative
4  petter mattei love time money visual stun film...  positive


In [20]:
# Creating Test/Train data for the reviews data
norm_train_reviews = imdb_data.review[:40000]
norm_test_reviews = imdb_data.review[40000:]

In [21]:
# Count vectorizer for bag of words
cv = CountVectorizer(min_df=0.0, max_df=1.0, binary=False, ngram_range=(1, 3))
# transformed train reviews
cv_train_reviews = cv.fit_transform(norm_train_reviews)
# transformed test reviews
cv_test_reviews = cv.transform(norm_test_reviews)

print("BOW_cv_train:", cv_train_reviews.shape)
print("BOW_cv_test:", cv_test_reviews.shape)

BOW_cv_train: (40000, 4166498)
BOW_cv_test: (10000, 4166498)


### Term Frequency-Inverse Document Frequency model (TFIDF)

TF-IDF is commonly used in information retrieval, text mining, and document classification tasks to identify key terms or features within documents and to rank their importance.


In [22]:
tv = TfidfVectorizer(min_df=0.0, max_df=1.0, use_idf=True, ngram_range=(1, 3))
# transformed train reviews
tv_train_reviews = tv.fit_transform(norm_train_reviews)
# transformed test reviews
tv_test_reviews = tv.transform(norm_test_reviews)
print("Tfidf_train:", tv_train_reviews.shape)
print("Tfidf_test:", tv_test_reviews.shape)

Tfidf_train: (40000, 4166498)
Tfidf_test: (10000, 4166498)


In [23]:
# labeling the sentient data
lb = LabelBinarizer()
# transformed sentiment data
sentiment_data = lb.fit_transform(imdb_data["sentiment"])
print(sentiment_data.shape)

(50000, 1)


In [24]:
# Spliting the sentiment data
train_sentiments = sentiment_data[:40000]
test_sentiments = sentiment_data[40000:]
print(train_sentiments.shape)
print(test_sentiments.shape)

(40000, 1)
(10000, 1)


### Multinomial Naive Bayes Model

In this section we will look at training a Bayes model.


In [25]:
# training the model
mnb = MultinomialNB()
# fitting the svm for bag of words
mnb_bow = mnb.fit(cv_train_reviews, train_sentiments)
print(mnb_bow)
# fitting the svm for tfidf features
mnb_tfidf = mnb.fit(tv_train_reviews, train_sentiments)
print(mnb_tfidf)

/Users/dsdevnull/Documents/githubProjects/sentiment-analysis-imdb/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB()


/Users/dsdevnull/Documents/githubProjects/sentiment-analysis-imdb/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB()


In [26]:
# Predicting the model for bag of words
mnb_bow_predict = mnb.predict(cv_test_reviews)
print(mnb_bow_predict)
# Predicting the model for tfidf features
mnb_tfidf_predict = mnb.predict(tv_test_reviews)
print(mnb_tfidf_predict)

[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]


In [27]:
# Accuracy score for bag of words
mnb_bow_score = accuracy_score(test_sentiments, mnb_bow_predict)
print("mnb_bow_score :", mnb_bow_score)
# Accuracy score for tfidf features
mnb_tfidf_score = accuracy_score(test_sentiments, mnb_tfidf_predict)
print("mnb_tfidf_score :", mnb_tfidf_score)

mnb_bow_score : 0.8585
mnb_tfidf_score : 0.8692


In [28]:
# Classification report for bag of words
mnb_bow_report = classification_report(
    test_sentiments, mnb_bow_predict, target_names=["Positive", "Negative"]
)
print(mnb_bow_report)
# Classification report for tfidf features
mnb_tfidf_report = classification_report(
    test_sentiments, mnb_tfidf_predict, target_names=["Positive", "Negative"]
)
print(mnb_tfidf_report)

              precision    recall  f1-score   support

    Positive       0.84      0.88      0.86      4993
    Negative       0.87      0.84      0.86      5007

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000

              precision    recall  f1-score   support

    Positive       0.87      0.87      0.87      4993
    Negative       0.87      0.87      0.87      5007

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



In [29]:
# confusion matrix for bag of words
cm_bow = confusion_matrix(test_sentiments, mnb_bow_predict, labels=[1, 0])
print(cm_bow)
# confusion matrix for tfidf features
cm_tfidf = confusion_matrix(test_sentiments, mnb_tfidf_predict, labels=[1, 0])
print(cm_tfidf)

[[4199  808]
 [ 607 4386]]
[[4367  640]
 [ 668 4325]]


### Linear Regression Model

This section will go over using a linear regression model


In [30]:
# training the model
lr = LogisticRegression(penalty="l2", max_iter=500, C=1, random_state=42)
# Fitting the model for Bag of words
lr_bow = lr.fit(cv_train_reviews, train_sentiments)
print(lr_bow)
# Fitting the model for tfidf features
lr_tfidf = lr.fit(tv_train_reviews, train_sentiments)
print(lr_tfidf)

/Users/dsdevnull/Documents/githubProjects/sentiment-analysis-imdb/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1, max_iter=500, random_state=42)


/Users/dsdevnull/Documents/githubProjects/sentiment-analysis-imdb/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1, max_iter=500, random_state=42)


In [31]:
# Predicting the model for bag of words
lr_bow_predict = lr.predict(cv_test_reviews)
print(lr_bow_predict)
##Predicting the model for tfidf features
lr_tfidf_predict = lr.predict(tv_test_reviews)
print(lr_tfidf_predict)

[0 0 0 ... 1 0 0]
[0 0 0 ... 1 0 0]


In [32]:
# Accuracy score for bag of words
lr_bow_score = accuracy_score(test_sentiments, lr_bow_predict)
print("lr_bow_score :", lr_bow_score)
# Accuracy score for tfidf features
lr_tfidf_score = accuracy_score(test_sentiments, lr_tfidf_predict)
print("lr_tfidf_score :", lr_tfidf_score)

lr_bow_score : 0.8238
lr_tfidf_score : 0.8559


In [33]:
# Classification report for bag of words
lr_bow_report = classification_report(
    test_sentiments, lr_bow_predict, target_names=["Positive", "Negative"]
)
print(lr_bow_report)

# Classification report for tfidf features
lr_tfidf_report = classification_report(
    test_sentiments, lr_tfidf_predict, target_names=["Positive", "Negative"]
)
print(lr_tfidf_report)

              precision    recall  f1-score   support

    Positive       0.81      0.84      0.83      4993
    Negative       0.84      0.80      0.82      5007

    accuracy                           0.82     10000
   macro avg       0.82      0.82      0.82     10000
weighted avg       0.82      0.82      0.82     10000

              precision    recall  f1-score   support

    Positive       0.87      0.83      0.85      4993
    Negative       0.84      0.88      0.86      5007

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



In [34]:
# confusion matrix for bag of words
cm_bow = confusion_matrix(test_sentiments, lr_bow_predict, labels=[1, 0])
print(cm_bow)
# confusion matrix for tfidf features
cm_tfidf = confusion_matrix(test_sentiments, lr_tfidf_predict, labels=[1, 0])
print(cm_tfidf)

[[4023  984]
 [ 778 4215]]
[[4390  617]
 [ 824 4169]]


# Predictions

We can see that we have about an 85% accuracy with this model! That is not bad, lets test some manually entered text.

Reviews that will be used in this section:

Breaking Bad is a radical type of television, and also a very strange kind of must-watch: a show that you dread and crave at the same time. - Positive

Taut, efficient and directed with a scalpel, Breaking Bad remains a marvel. - Positive

A freak with a falsetto voice running around acting stupid, come on. After watching three episodes I still cannot say exactly what the show is about and I found that I was looking forward to the commercials and the sweet release of the end credits. I don't know what they were taking at Nickelodeon when they decided to put this show on TV but I want some. Instead of airing episodes of Fred they should just send out whatever it is they are taking so we all feel good. I understand that Fred was a YouTube Star. Perhaps Fred should return to the internet and Nickelodeon should find some shows that appeal to the entire family. Nickelodeon said they were looking into why Disney beat them in the ratings; the answer is FRED. - Negative

This is one of the worst shows ever to hit T.V. besides Little Miss Perfect. I could not tolerate even after 2 minutes of this garbage. This show is about moms teaching their kids how to dress up like a prostitute for pageants that are below the age of about 8! This show is supposed to be about toddlers, why do they got seven year olds in there? This title doesn't even make sense! The kids are ugly, the moms are pressuring their kids to do challenging exercises, why can't they just treat their kid as if they were a kid? I can't believe what America has come to! Seriously, this series has got to be fake! - Negative


In [35]:
review_list = [
    "Breaking Bad is a radical type of television, and also a very strange kind of must-watch: a show that you dread and crave at the same time.",
    "Taut, efficient and directed with a scalpel, Breaking Bad remains a marvel.",
    "A freak with a falsetto voice running around acting stupid, come on. After watching three episodes I still cannot say exactly what the show is about and I found that I was looking forward to the commercials and the sweet release of the end credits. I don't know what they were taking at Nickelodeon when they decided to put this show on TV but I want some. Instead of airing episodes of Fred they should just send out whatever it is they are taking so we all feel good. I understand that Fred was a YouTube Star. Perhaps Fred should return to the internet and Nickelodeon should find some shows that appeal to the entire family. Nickelodeon said they were looking into why Disney beat them in the ratings; the answer is FRED.",
    "This is one of the worst shows ever to hit T.V. besides Little Miss Perfect. I could not tolerate even after 2 minutes of this garbage. This show is about moms teaching their kids how to dress up like a prostitute for pageants that are below the age of about 8! This show is supposed to be about toddlers, why do they got seven year olds in there? This title doesn't even make sense! The kids are ugly, the moms are pressuring their kids to do challenging exercises, why can't they just treat their kid as if they were a kid? I can't believe what America has come to! Seriously, this series has got to be fake!",
]

manual_reviews = pd.DataFrame(review_list, columns=["manually_curated_reviews"])

# Text Preprocessing
manual_reviews["manually_curated_reviews"] = manual_reviews[
    "manually_curated_reviews"
].apply(clean_html)
manual_reviews["manually_curated_reviews"] = manual_reviews[
    "manually_curated_reviews"
].apply(remove_spec_char)
manual_reviews["manually_curated_reviews"] = manual_reviews[
    "manually_curated_reviews"
].str.lower()
manual_reviews["manually_curated_reviews"] = manual_reviews[
    "manually_curated_reviews"
].apply(simple_stemmer)
manual_reviews["manually_curated_reviews"] = manual_reviews[
    "manually_curated_reviews"
].apply(remove_stop_words)

print(manual_reviews)

                            manually_curated_reviews
0  break bad radic type televis also veri strang ...
1  taut effici direct scalpel break bad remain ma...
2  freak falsetto voic run around act stupid come...
3  thi one worst show ever hit tv besid littl mis...


In [36]:
# Linear Regression Manual Testing
lr_bow_prediction_manual = lr_bow.predict(
    cv.transform(manual_reviews["manually_curated_reviews"])
)
lr_tfidf_prediction_manual = lr.predict(
    cv.transform(manual_reviews["manually_curated_reviews"])
)
print(lr_bow_prediction_manual)
print(lr_tfidf_prediction_manual)

[0 0 0 0]
[0 0 0 0]


## Linear Regression Assessment

I am surprised by this, after doing some more testing though, the reason we are getting all negative results is because the title of the show "Breaking Bad" is present in the review, this lets the model believe that the review itself has a negative sentiment. I tested this by removing 'Bad' from the review and it ended up being the case.

One future enhancement that can be made, is checking if the name of the show is present in the review, if it is, we can remove it. This can be something done once webscrapping is implemented and I can get my own reviews.


In [37]:
# Multinomial Naive Bayes Model Manual Testing
mnb_bow_prediction_testing = mnb.predict(
    cv.transform(manual_reviews["manually_curated_reviews"])
)
mnb_tfidf_prediction_testing = mnb.predict(
    cv.transform(manual_reviews["manually_curated_reviews"])
)

print(mnb_bow_prediction_testing)
print(mnb_tfidf_prediction_testing)

[0 1 0 0]
[0 1 0 0]


## Multinomial Naive Bayes Model Assessment

I am pleasanty surprised that this worked out better than the linear regression model. Still seemed to have failed in the first review because of the title being breaking bad, but I am not sure right now. Further evaluation will need to be made.


In [39]:
# Exporting the models to be added to hugging face or implemented in other projects!
import joblib

joblib.dump(lr_bow, "linear_regression_bow_sentiment_analysis.pkl")
joblib.dump(lr_tfidf, "linear_regression_tfidf_sentiment_analysis.pkl")
joblib.dump(mnb_bow, "mn_bayes_bow_sentiment_analysis.pkl")
joblib.dump(mnb_tfidf, "mn_bays_tfidf_sentiment_analysis.pkl")

['mn_bays_tfidf_sentiment_analysis.pkl']